In [ ]:
ls /kaggle/input/physionet-ecg-image-digitization/train/1006427285

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv('/kaggle/input/physionet-ecg-image-digitization/train.csv')
test = pd.read_csv('/kaggle/input/physionet-ecg-image-digitization/test.csv')
submission = pd.read_parquet('/kaggle/input/physionet-ecg-image-digitization/sample_submission.parquet')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
def get_image_type(filename):
    """Determine image type based on filename"""
    type_mapping = {
        '0001': 'original_color',
        '0003': 'printed_scanned_color', 
        '0004': 'printed_scanned_bw',
        '0005': 'mobile_photo_color',
        '0006': 'mobile_photo_screen',
        '0009': 'stained_soaked',
        '0010': 'extensive_damage',
        '0011': 'mold_color',
        '0012': 'mold_bw'
    }

    image_id = filename.split('-')[1].split('.')[0]
    return type_mapping.get(image_id, 'unknown')

**Dataset Transformer class**

In [ ]:
# ============================================================
# DATASET : Image → 12 Leads
# ============================================================
import random
class MultiLeadECGDataset(Dataset):
    """
    Dataset pour entraîner le modèle :
    Input : Image ECG
    Output : 12 signaux (I, II, III, aVR, aVL, aVF, V1-V6)
    """
    
    def __init__(self, train_csv, train_dir, img_size=(512, 512)):
        """
        Args:
            train_csv: Fichier train.csv avec id, fs, sig_len
            train_dir: Dossier contenant les images et signaux
            img_size: Taille de redimensionnement des images
        """
        self.metadata = pd.read_csv(train_csv)
        self.train_dir = train_dir
        self.img_size = img_size
        self.lead_names = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
        self.image_used_types = ['original_color', 'printed_scanned_color', 'mobile_photo_color']
        
        # Déterminer la longueur maximale des signaux
        self.max_signal_length = self._get_max_signal_length()
        
    def _get_max_signal_length(self):
        """Trouve la longueur maximale des signaux dans le dataset"""
        max_len = 0
        for _, row in self.metadata.iterrows():
            csv_path = os.path.join(self.train_dir, str(row['id']), f"{row['id']}.csv")
            if os.path.exists(csv_path):
                df = pd.read_csv(csv_path)
                max_len = max(max_len, len(df))
        return max_len
    
    def __len__(self):
        return len(self.metadata)
    
    def _get_images_path(self, sample_id):
        """Trouve les images disponible pour cet ID"""
        base_dir = os.path.join(self.train_dir, str(sample_id))
        images_path = []
        for filename in os.listdir(base_dir):
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            image_type = get_image_type(filename)
            if image_type in self.image_used_types:
                image_path = os.path.join(base_dir, filename)
                images_path.append(image_path)
        return images_path

    
    def __getitem__(self, idx):
        sample_id = self.metadata.iloc[idx]['id']
        fs = self.metadata.iloc[idx]['fs']
        sig_len = self.metadata.iloc[idx]['sig_len']
        
        # 1. Charger l'image
        img_paths = self._get_images_path(sample_id)
        if not img_paths:
            # Image non trouvée, retourner des données vides
            image = np.zeros((3, *self.img_size), dtype=np.float32)
            signals = np.zeros((12, self.max_signal_length), dtype=np.float32)
            mask = np.zeros((12, self.max_signal_length), dtype=np.float32)
            return {
                'image': torch.from_numpy(image),
                'signals': torch.from_numpy(signals),
                'mask': torch.from_numpy(mask),
                'id': sample_id
            }
        img_idx = random.randint(0, len(img_paths) -1)
        img_path = img_paths[img_idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, self.img_size)
        
        # Normalisation [0, 1]
        image = image.astype(np.float32) / 255.0
        
        # Transpose pour PyTorch (H, W, C) → (C, H, W)
        image = image.transpose(2, 0, 1)
        
        # 2. Charger les signaux (labels)
        csv_path = os.path.join(self.train_dir, str(sample_id), f"{sample_id}.csv")
        signal_df = pd.read_csv(csv_path)
        
        # Initialiser les signaux et le masque
        signals = np.zeros((12, self.max_signal_length), dtype=np.float32)
        mask = np.zeros((12, self.max_signal_length), dtype=np.float32)
        
        # Remplir chaque lead
        for i, lead in enumerate(self.lead_names):
            if lead in signal_df.columns:
                lead_signal = signal_df[lead].values
                
                # Supprimer les NaN
                valid_indices = ~np.isnan(lead_signal)
                valid_signal = lead_signal[valid_indices]
                
                if len(valid_signal) > 0:
                    # Stocker le signal
                    length = min(len(valid_signal), self.max_signal_length)
                    signals[i, :length] = valid_signal[:length]
                    mask[i, :length] = 1.0  # Marquer comme valide
        
        return {
            'image': torch.from_numpy(image),
            'signals': torch.from_numpy(signals),
            'mask': torch.from_numpy(mask),
            'id': sample_id,
            'fs': fs,
            'sig_len': sig_len
        }

**Model Definition**

In [ ]:
# ============================================================
# MODÈLE U-NET MULTI-OUTPUT
# ============================================================

class UNetEncoder(nn.Module):
    """Encoder U-Net pour extraction de features"""
    
    def __init__(self):
        super().__init__()
        
        # Encoder blocks
        self.enc1 = self._make_encoder_block(3, 64)
        self.enc2 = self._make_encoder_block(64, 128)
        self.enc3 = self._make_encoder_block(128, 256)
        self.enc4 = self._make_encoder_block(256, 512)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        # Bottleneck
        self.bottleneck = self._make_encoder_block(512, 1024)
        
    def _make_encoder_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        # Encoder path avec skip connections
        e1 = self.enc1(x)        # [B, 64, H, W]
        e2 = self.enc2(self.pool(e1))  # [B, 128, H/2, W/2]
        e3 = self.enc3(self.pool(e2))  # [B, 256, H/4, W/4]
        e4 = self.enc4(self.pool(e3))  # [B, 512, H/8, W/8]
        
        bottleneck = self.bottleneck(self.pool(e4))  # [B, 1024, H/16, W/16]
        
        return bottleneck, [e4, e3, e2, e1]



**Signal decoder**

In [ ]:

class MultiLeadDecoder(nn.Module):
    """
    Decoder qui produit 12 signaux 1D
    """
    
    def __init__(self, signal_length=5000):
        super().__init__()
        self.signal_length = signal_length
        
        # Global pooling pour réduire la dimensionnalité spatiale
        self.gap = nn.AdaptiveAvgPool2d(1)
        
        # Shared layers
        self.shared_fc = nn.Sequential(
            nn.Linear(1024, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(2048, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2)
        )
        
        # 12 têtes de sortie (une pour chaque lead)
        self.lead_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(4096, 2048),
                nn.ReLU(inplace=True),
                nn.Linear(2048, signal_length)
            ) for _ in range(12)
        ])
        
    def forward(self, x):
        # x: [B, 1024, H, W]
        
        # Global pooling
        x = self.gap(x)  # [B, 1024, 1, 1]
        x = x.view(x.size(0), -1)  # [B, 1024]
        
        # Shared features
        shared_features = self.shared_fc(x)  # [B, 4096]
        
        # Prédire chaque lead séparément
        lead_outputs = []
        for head in self.lead_heads:
            lead_signal = head(shared_features)  # [B, signal_length]
            lead_outputs.append(lead_signal)
        
        # Empiler les 12 leads
        outputs = torch.stack(lead_outputs, dim=1)  # [B, 12, signal_length]
        
        return outputs


class MultiLeadECGModel(nn.Module):
    """Modèle complet : Image → 12 signaux"""
    
    def __init__(self, signal_length=5000):
        super().__init__()
        self.encoder = UNetEncoder()
        self.decoder = MultiLeadDecoder(signal_length)
        
    def forward(self, image):
        # Encoder
        features, skips = self.encoder(image)
        
        # Decoder
        signals = self.decoder(features)  # [B, 12, signal_length]
        
        return signals

In [ ]:

# ============================================================
# LOSS FUNCTION AVEC MASQUE
# ============================================================

class MaskedMSELoss(nn.Module):
    """MSE Loss qui ignore les valeurs NaN (masquées)"""
    
    def __init__(self):
        super().__init__()
    
    def forward(self, predictions, targets, mask):
        """
        Args:
            predictions: [B, 12, signal_length]
            targets: [B, 12, signal_length]
            mask: [B, 12, signal_length] - 1 pour valide, 0 pour NaN
        """
        # Calculer l'erreur seulement sur les valeurs valides
        squared_error = (predictions - targets) ** 2
        masked_error = squared_error * mask
        
        # Moyenne sur les valeurs valides
        num_valid = mask.sum()
        if num_valid > 0:
            loss = masked_error.sum() / num_valid
        else:
            loss = torch.tensor(0.0, device=predictions.device)
        
        return loss

In [ ]:

# ============================================================
# TRAINING FUNCTION
# ============================================================

def train_multilead_model(model, train_loader, val_loader, num_epochs=50, device='cuda'):
    """Entraîne le modèle multi-lead"""
    
    criterion = MaskedMSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)
    
    history = {'train_loss': [], 'val_loss': []}
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # ===== TRAINING =====
        model.train()
        train_loss = 0.0
        
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for batch in pbar:
            images = batch['image'].to(device)
            signals = batch['signals'].to(device)
            mask = batch['mask'].to(device)
            
            # Forward
            optimizer.zero_grad()
            predictions = model(images)
            
            # Ajuster la longueur si nécessaire
            min_len = min(predictions.shape[2], signals.shape[2])
            predictions = predictions[:, :, :min_len]
            signals = signals[:, :, :min_len]
            mask = mask[:, :, :min_len]
            
            # Loss
            loss = criterion(predictions, signals, mask)
            
            # Backward
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            pbar.set_postfix({'loss': loss.item()})
        
        train_loss /= len(train_loader)
        history['train_loss'].append(train_loss)
        
        # ===== VALIDATION =====
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]')
            for batch in pbar:
                images = batch['image'].to(device)
                signals = batch['signals'].to(device)
                mask = batch['mask'].to(device)
                
                predictions = model(images)
                
                min_len = min(predictions.shape[2], signals.shape[2])
                predictions = predictions[:, :, :min_len]
                signals = signals[:, :, :min_len]
                mask = mask[:, :, :min_len]
                
                loss = criterion(predictions, signals, mask)
                val_loss += loss.item()
                pbar.set_postfix({'loss': loss.item()})
        
        val_loss /= len(val_loader)
        history['val_loss'].append(val_loss)
        
        # Scheduler
        scheduler.step(val_loss)
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
            }, 'best_multilead_ecg_model.pth')
            print(f'✓ Nouveau meilleur modèle ! Val Loss: {val_loss:.6f}')
        
        print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}\n')
    
    return history


In [ ]:


# ============================================================
# INFERENCE POUR TEST SET
# ============================================================

def predict_test_set(model, test_df, test_dir, device='cuda', img_size=(512, 512)):
    """Prédire sur le test set"""
    
    model.eval()
    predictions = {}
    
    lead_names = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    
    with torch.no_grad():
        for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc='Predicting'):
            image_id = row['id']
            requested_lead = row['lead']
            target_length = row['number_of_rows']
            
            # Charger l'image
            img_path = os.path.join(test_dir, f"{image_id}.png")
            
            if not os.path.exists(img_path):
                # Image non trouvée
                predictions[(image_id, requested_lead)] = np.zeros(target_length)
                continue
            
            # Préprocessing
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, img_size)
            image = image.astype(np.float32) / 255.0
            image = image.transpose(2, 0, 1)
            
            # Convertir en tensor
            image_tensor = torch.from_numpy(image).unsqueeze(0).to(device)
            
            # Prédire
            all_signals = model(image_tensor)  # [1, 12, signal_length]
            
            # Extraire le lead demandé
            lead_idx = lead_names.index(requested_lead)
            signal = all_signals[0, lead_idx, :].cpu().numpy()
            
            # Rééchantillonner à la longueur demandée
            if len(signal) != target_length:
                x_old = np.linspace(0, 1, len(signal))
                x_new = np.linspace(0, 1, target_length)
                signal = np.interp(x_new, x_old, signal)
            
            predictions[(image_id, requested_lead)] = signal
    
    return predictions

In [ ]:

# ============================================================
# PIPELINE COMPLET
# ============================================================

def main_pipeline():
    """Pipeline complet d'entraînement"""
    
    print("="*70)
    print("🚀 TRAINING MULTI-LEAD ECG MODEL")
    print("="*70)
    
    # Configuration
    BATCH_SIZE = 4
    NUM_EPOCHS = 50
    IMG_SIZE = (512, 512)
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    print(f"\n📱 Device: {DEVICE}")
    print(f"🖼️  Image size: {IMG_SIZE}")
    print(f"📦 Batch size: {BATCH_SIZE}")
    
    # Dataset
    print("\n📊 Chargement des données...")
    dataset = MultiLeadECGDataset(
        train_csv='/kaggle/input/physionet-ecg-image-digitization/train.csv',
        train_dir='/kaggle/input/physionet-ecg-image-digitization/train',
        img_size=IMG_SIZE
    )
    
    print(f"   Max signal length: {dataset.max_signal_length}")
    
    # Split
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
    print(f"   Train: {len(train_dataset)} samples")
    print(f"   Val: {len(val_dataset)} samples")
    
    # Modèle
    print("\n🏗️  Construction du modèle...")
    model = MultiLeadECGModel(signal_length=dataset.max_signal_length).to(DEVICE)
    
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"   Paramètres: {num_params:,}")
    
    # Training
    print("\n🎯 Début du training...\n")
    history = train_multilead_model(model, train_loader, val_loader, num_epochs=NUM_EPOCHS, device=DEVICE)
    
    # Plot history
    plt.figure(figsize=(10, 5))
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training History')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('training_history.png')
    plt.show()
    
    print("\n✅ Training terminé !")
    print("📁 Modèle sauvegardé : best_multilead_ecg_model.pth")

# Pour lancer
# main_pipeline()

Training

In [ ]:
# main_pipeline()

In [ ]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread("/kaggle/input/physionet-ecg-image-digitization/train/1006867983/1006867983-0001.png")
b, g, r = cv2.split(img)

plt.figure(figsize=(10,3))
plt.subplot(131); plt.imshow(r, cmap='gray'); plt.title("Canal Rouge")
plt.subplot(132); plt.imshow(g, cmap='gray'); plt.title("Canal Vert")
plt.subplot(133); plt.imshow(b); plt.title("Canal Bleu")
plt.show()

cv2.imwrite('image.png', b)



In [ ]:
def get_image_type(filename):
    """Determine image type based on filename"""
    type_mapping = {
        '0001': 'original_color',
        '0003': 'printed_scanned_color', 
        '0004': 'printed_scanned_bw',
        '0005': 'mobile_photo_color',
        '0006': 'mobile_photo_screen',
        '0009': 'stained_soaked',
        '0010': 'extensive_damage',
        '0011': 'mold_color',
        '0012': 'mold_bw'
    }

    image_id = filename.split('-')[1].split('.')[0]
    return type_mapping.get(image_id, 'unknown')

In [ ]:
import os
original_count = 0
total_count = 0
def check_all_uniform(train_directory):
    global original_count, total_count
    for root, _, files in os.walk(train_directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', 'JPG')):
                image_type = get_image_type(file)
                if image_type == 'original_color':
                    original_count += 1
                total_count += 1

check_all_uniform('/kaggle/input/physionet-ecg-image-digitization/train')
print('original_count', original_count, 'total_count', total_count)

In [ ]:
import pandas as pd

train = pd.read_csv('/kaggle/input/physionet-ecg-image-digitization/train.csv')
test = pd.read_csv('/kaggle/input/physionet-ecg-image-digitization/test.csv')
submission = pd.read_parquet('/kaggle/input/physionet-ecg-image-digitization/sample_submission.parquet')

In [ ]:
print(train.info())

In [ ]:
print(test.info())

In [ ]:
label_1_path = "/kaggle/input/physionet-ecg-image-digitization/train/1006867983/1006867983.csv"
label = pd.read_csv(label_1_path)
print(label.head())

In [ ]:
def _get_images_path(sample_id):
    """Trouve les images disponible pour cet ID"""
    train_dir = '/kaggle/input/physionet-ecg-image-digitization/train'
    base_dir = os.path.join(train_dir, str(sample_id))
    images_path = []
    for filename in os.listdir(base_dir):
        
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        print(filename)

In [ ]:

_get_images_path('1006427285')

In [ ]:
ls /kaggle/working

In [ ]:
import torch

# Charger le modèle
model = MultiLeadECGModel(signal_length=10250)
model = model.to('cuda')
checkpoint = torch.load("/kaggle/working/best_multilead_ecg_model.pth", map_location='cuda')

# Charger les poids
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
def create_submission_from_multilead(predictions, test_df, filename='submission.csv'):
    """
    Crée le fichier de soumission au format requis
    
    Format attendu :
    id, value
    {image_id}_{row_idx}_{lead}, {signal_value}
    
    Args:
        predictions: dict {(image_id, lead): signal_array}
        test_df: DataFrame du test
        filename: Nom du fichier de sortie
    """
    submission_data = []
    
    for _, row in test_df.iterrows():
        image_id = row['id']
        lead = row['lead']
        n_rows = row['number_of_rows']
        
        # Récupérer le signal prédit
        signal = predictions.get((image_id, lead), np.zeros(n_rows))
        
        # Créer les entrées pour chaque point du signal
        for i in range(n_rows):
            submission_id = f"{image_id}_{i}_{lead}"
            value = float(signal[i])
            
            submission_data.append({
                'id': submission_id,
                'value': value
            })
    
    # Créer le DataFrame
    submission_df = pd.DataFrame(submission_data)
    
    # Statistiques
    print(f"\n📊 Statistiques de la soumission :")
    print(f"   Total d'entrées : {len(submission_df):,}")
    print(f"   Range : [{submission_df['value'].min():.4f}, {submission_df['value'].max():.4f}]")
    print(f"   Moyenne : {submission_df['value'].mean():.4f}")
    print(f"   Std : {submission_df['value'].std():.4f}")
    
    # Vérifier NaN
    if submission_df['value'].isna().any():
        print(f"⚠️  {submission_df['value'].isna().sum()} valeurs NaN, remplacement par 0")
        submission_df['value'].fillna(0, inplace=True)
    
    # Sauvegarder
    submission_df.to_csv(filename, index=False)
    print(f"\n✅ Soumission sauvegardée : {filename}")
    print(f"📏 Shape : {submission_df.shape}")
    
    return submission_df



In [ ]:
device = 'cuda'
def submission_from_pretrained():
    """Créer soumission depuis un modèle pré-entraîné"""
    
    print("="*70)
    print("🎯 SOUMISSION DEPUIS MODÈLE PRÉ-ENTRAÎNÉ")
    print("="*70)
    
    # Charger test
    test = pd.read_csv('/kaggle/input/physionet-ecg-image-digitization/test.csv')
    test_dir = '/kaggle/input/physionet-ecg-image-digitization/test/'
    
    # Prédire
    predictions = predict_test_set(model, test, test_dir, device=device)
    
    # Créer soumission
    submission = create_submission_from_multilead(predictions, test, 'submission.csv')
    
    print("\n✅ Soumission prête !")
    
    return submission

In [ ]:
submission_from_pretrained()

In [ ]:
print(next(model.parameters()).device)